In [ ]:
import pandas as pd
import numpy as np
import os
import xarray as xr 
import plotly.express as px

os.getcwd()
DIR = "X:\\user\\zapatacasv\\sdg7\\scripts"   # Set the running directory here
os.chdir(DIR)
#import data_import as dat 
all = xr.open_dataset('..\\output\\gph_SSP2\\gph_popg_world_SSP2.nc')
import test as gph
gph.popg = all.tpop

del all

In [ ]:
#Opening least cost technology maps under all scenarios
#list of scenarios
scen = ['SSP2','SSP2-DLS',  'PEP-2C-AdvPE', 'PEP-2C-AdvPE-DLS']
fn = ['techid_univ', 'techid_bl']
path = []
fn2 = []
for s in scen:
    for f in fn:
        arr = '..\\output\\%s\\%s.nc' %(s, f)
        path.append(arr)
        arr = f + '-' + s #s #
        fn2.append(arr)
techid = xr.combine_by_coords([xr.open_dataarray(path[i]).to_dataset(name = fn2[i] ) for i in range(len(path))], combine_attrs = "override")


In [ ]:
#for filtering and changing scenario names
fn = fn2
fn2 = ['techid_univ-SSP2',
 'techid_bl-SSP2',
 'techid_univ-SSP2-DLS',
 'techid_univ-PEP-2C-AdvPE',
 #'techid_bl-PEP-2C-AdvPE',
 'techid_univ-PEP-2C-AdvPE-DLS']

scen = {'techid_univ-SSP2' : 'SSP2-UA',
 'techid_bl-SSP2' :  'SSP2-BL',
 'techid_univ-SSP2-DLS' :  'SSP2-UA-DLS',
 'techid_univ-PEP-2C-AdvPE' :  'SSP2-2C-UA',
 #'techid_bl-PEP-2C-AdvPE' : 'SSP2-2C-BL',
 'techid_univ-PEP-2C-AdvPE-DLS' : 'SSP2-2C-UA-DLS'}

techid = techid[fn2]

In [ ]:
#Plotting distribution of least cost options in the population for achieving universal access
year = gph.popg.year.data

arr = techid
arr = xr.where(arr == 1, np.nan, arr)
arr['pop'] = gph.popg.sel(year = 2030) 
df1 = pd.DataFrame([])
for r in [4,6, 8.0, 9.0,10,21,22,25,26, 18, 3]: #24
    for f in fn2:
        df = arr.where(arr.imager == r)
        df.coords[f] = df[f]
        df = df.pop.groupby(f).sum().drop(['spatial_ref',  'dos', 'year', 'seis', 'tres']).to_dataframe(name = ('pop'))
        df['region'] = r
        df['scenario'] = scen[f]
        df['pop_share'] = df['pop'] / df['pop'].sum()
        df1 = pd.concat([df1,df])#df1.append(df)
        #df1


df2 = df1.reset_index(names = 'leastcostID_enteryear_univ')
#df2 = df2[df2["region"].isin([4, 8.0, 9.0,21,22,25,26, 18])]  #, 3, 10, 6,
df2 = df2[df2["region"].isin([4, 8.0, 9.0,21,22,25,26, 18, 3, 10, 6])]  #, 3, 10, 6,
df2 = df2[df2["scenario"].isin(scen.values())]
df2 = df2.replace({'leastcostID_enteryear_univ': {  2:'CG-densification', 3: 'CG-extension', 4: 'MG-PV', 5: 'MG-diesel', 6: 'MG-wind', 7:'MG-hydro' , 8:'MG-PV-diesel', 9:'MG-wind-diesel', 10:'SA-PV', 11: "SA-diesel"}}) #1: 'already access',
df2 = df2.replace({'region':  {4: 'Rest Central America', 6:'Rest South America', 8:'Western Africa', 9: 'Eastern Africa', 10: 'South Africa', 21: 'Southeastern Asia', 22: 'Indonesia +', 24:'Oceania' , 25:'Rest South Asia', 26:'Rest South Africa', 18: 'India', 3: 'Mexico'}})  #{ 8:'Western Africa', 9: 'Eastern Africa', 10: 'South Africa', 26:'Rest SSA'}}) #{6:'Rest South America', 8:'Western Africa', 9: 'Eastern Africa', 10: 'South Africa', 21: 'Southeastern Asia', 22: 'Indonesia +', 24:'Oceania' , 25:'Rest S.Asia', 26:'Rest S.Africa'}})





In [ ]:
df2.to_csv('..\\output\\tech_shares_over_pop2.csv')

In [ ]:
# #Analyzing difference in LCOE between scenarios with and without mitigation
# df = techid.where(techid>3)

# #df['techid_univ-PEP-2C-AdvPE'] = df['techid_univ-PEP-2C-AdvPE'].where((df['techid_univ-PEP-2C-AdvPE'] != 5) & (df['techid_univ-PEP-2C-AdvPE'] != 11))
# df['techid_univ-SSP2' ] = df['techid_univ-SSP2' ].where((df['techid_univ-SSP2' ] == 5) | (df['techid_univ-SSP2' ] == 11) | (df['techid_univ-SSP2' ] == 8) | (df['techid_univ-SSP2' ] == 9))

# lcoe_mit = xr.open_dataarray('..\\output\\PEP-2C-AdvPE\\leastcost_LCOE_enteryear_univ.nc')
# lcoe = xr.open_dataarray('..\\output\\SSP2\\leastcost_LCOE_enteryear_univ.nc')

# delta_LCOE = (lcoe_mit.where(df['techid_univ-SSP2' ] >0) - lcoe.where(df['techid_univ-SSP2' ] >0))*100/lcoe


# #delta_LCOE.plot.hist(bins = 100)
# delta_LCOE.where(delta_LCOE == 0).count()

In [ ]:

fig = px.bar(df2,x='scenario', y='pop',  color="leastcostID_enteryear_univ", facet_col='region', facet_col_wrap=4 , 
             category_orders={"scenario" : [ 'SSP2-BL', 'SSP2-UA', 'SSP2-2C-UA', 'SSP2-UA-DLS' , 'SSP2-2C-UA-DLS'] , 
                              "leastcostID_enteryear_univ" : [ 'CG-densification', 'CG-extension',   'MG-wind', 'MG-PV','MG-hydro' ,   'MG-PV-diesel', 'MG-wind-diesel','MG-diesel', "SA-diesel", 'SA-PV'], 
                              "region": [ 'Western Africa',  'Eastern Africa', 'Rest South Africa', 'Rest South Asia', 'Southeastern Asia', 'India','Indonesia +',       'Rest Central America']},  # 'Rest South America','Mexico', 'South Africa',
                              color_discrete_sequence=['#636EFA', '#AB63FA',   '#00CC96',  'rgb(255,237,111)','#19D3F3',    '#FF7F0E',  '#BCBD22', '#7F7F7F', "brown",'#FECB52'],
             labels = {"leastcostID_enteryear_univ" : "Legend:" , 'pop': 'Population', 'scenario': ' '},
             facet_row_spacing=0.25, facet_col_spacing=0.06) #

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
#fig.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True)) 
fig.update_layout(font = dict(size = 15), height = 600, width = 1300,  paper_bgcolor = "white", plot_bgcolor = "rgba(0,0,0,0)", legend=dict(orientation="h", yanchor="bottom", xanchor="left", y = -0.25, x = -0.05))
fig.update_xaxes( showticklabels=True, tickfont =dict(size = 13))
fig.update_yaxes( matches=None, showticklabels=True, tickfont =dict(size = 13)) #

In [ ]:
df2.to_csv('..\\output\\tech_shares_over_pop.csv')

In [ ]:
#Cleaning data from countries that achieved UA when running for world
#countries with UA by 2020
path = "..\\data\\UA_countries_2020.csv"
df2 =  pd.read_csv(path,  delimiter = ',')
#loading country  mask and removing countries that achieved UA
df = xr.open_dataset('..\\data\\country_5min.nc')
df = df.assign_coords(x = ("x", gph.popg.x.values))
df = df.assign_coords(y = ("y", gph.popg.y.values)) 
gph.popg.coords['country'] = xr.where(df.country.isin(df2.id), 1000, np.nan) #to make a shade on locations that achieved UA
gph.popg['country'] = gph.popg.country.where((gph.popg.country != 255) & (gph.popg.country != 8) & (gph.popg.country != 88) & (gph.popg.country != 214)) #remove oceans and poles for country code (8 here is not E Africa), 214 for Svalvard in north pole

#gph.popg = gph.popg.where(gph.popg['country']>=0)


In [ ]:
a = 'techid_univ-SSP2' #'techid_univ-PEP-2C-AdvPE' #'techid_univ-SSP2-DLS' #'techid_univ-PEP-2C-AdvPE' # #'techid_univ-SSP2-DLS'
techid[a].groupby(techid[a]).count()  #'techid_univ-SSP2' 'techid_univ-PEP-2C-AdvPE-DLS' 'techid_univ-SSP2-DLS', 'techid_univ-PEP-2C-AdvPE'

In [ ]:

#plotting least cost option for connection (central gid, mini grid, stad alone)
system = xr.full_like(techid[a] , np.nan)  
system = xr.where(techid[a]==1, np.nan, system) 
system = xr.where(( techid[a]==2) | (techid[a]==3), 1, system)
system = xr.where(( techid[a]>3)&(techid[a]<10),2, system)
system = xr.where(( techid[a]>9),3, system)
#system = system.where(gph.popg.tpop.sel(year = 2020) > 1) #to remove locations where there is no population

arr = system


import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:


# First we specify Coordinate Refference System for Map Projection
# We will use Mercator, which is a cylindrical, conformal projection. 
# It has bery large distortion at high latitudes, cannot 
# fully reach the polar regions.
projection = ccrs.PlateCarree() #ccrs.Mollweide()

# Specify CRS, that will be used to tell the code, where should our data be plotted
crs = ccrs.PlateCarree()

# Now we will create axes object having specific projection 
plt.figure(dpi=150)
ax = plt.axes(projection=projection, frameon=True)

# # Draw gridlines in degrees over Mollweide map
# gl = ax.gridlines(crs=crs, draw_labels=True,
#                   linewidth=.6, color='gray', alpha=0.5, linestyle='-.')
# gl.ylabels_right= False
# gl.ylabels_left= True
# gl.xlabels_top = False
# gl.xlabels_bottom = True
# #gl.xlocator = mticker.FixedLocator([-160,  0, 160])
# gl.xformatter = LONGITUDE_FORMATTER
# gl.yformatter = LATITUDE_FORMATTER
# gl.xlabel_style = {"size" : 7}
# gl.ylabel_style = {"size" : 7}
# gl.ylocator = mticker.FixedLocator([-20, 0, 20])
# To plot borders and coastlines, we can use cartopy feature
import cartopy.feature as cf
ax.coastlines(lw=0.3) #add_feature(cf.COASTLINE.with_scale("50m"), lw=0.3)
#ax.add_feature(cf.BORDERS.with_scale("50m"), lw=0.3)

# # Now, we will specify extent of our map in minimum/maximum longitude/latitude
# # Note that these values are specified in degrees of longitude and degrees of latitude
# # However, we can specify them in any crs that we want, but we need to provide appropriate
# # crs argument in ax.set_extent
lon_min = -179
lon_max = 179
lat_min = -40 #-70 #-44
lat_max = 60# 80 #44
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=crs)

cax = arr.plot(ax = ax,  add_colorbar=False, transform=ccrs.PlateCarree(), levels = [1,2,3,4]) #
gph.popg['country'].plot( ax = ax,  add_colorbar=False, transform=ccrs.PlateCarree(), levels = 1000, colors = 'lightgrey')    
plt.ylabel('latitude', fontsize=20)
plt.title(' ', fontsize=20)
cbar = plt.colorbar(cax, ticks=[1.5, 2.5, 3.5],  orientation='horizontal', spacing = 'uniform', shrink = 0.5) #colorbar
cbar.ax.set_xticklabels(['Central grid extension T&D', 'Mini grid', 'Stand-alone'],   fontsize=10)
#cbar.set_label('Map legend')
#plt.title(f"Temperature anomaly over Europe in {dataset.valid_time.dt.strftime('%B %Y').values}")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.add_feature(cf.OCEAN)
plt.show()